In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import implicit

In [2]:
df = pd.read_pickle('../../datasets/clean_df.csv', compression='zip')

In [15]:
df_orig.head()

,user,song,count,Song - Artist,user_id,song_id
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOWEZSI12A81C21CE6,1,Tu Quieres Volver - Gipsy Kings,300402,3214
1,833c530ecda3d99deb8395f70400aa3999783d91,SOWEZSI12A81C21CE6,2,Tu Quieres Volver - Gipsy Kings,214106,3214
2,d6c5bd2b570b4faf8964d7ed04f3392ff505d2be,SOWEZSI12A81C21CE6,1,Tu Quieres Volver - Gipsy Kings,350627,3214
3,724534729c9f5dc72a009269c2c225883e4775d2,SOWEZSI12A81C21CE6,1,Tu Quieres Volver - Gipsy Kings,186356,3214
4,ee7aa84c164038c963cfd02a7e52a5598aa470c3,SOWEZSI12A81C21CE6,2,Tu Quieres Volver - Gipsy Kings,389460,3214


In [16]:
len(users)

418252

In [17]:
len(songs)

3675

In [3]:
# Convert user id and song id to numerical ids
df['user_id'] = df['user'].astype('category').cat.codes
df['song_id'] = df['song'].astype('category').cat.codes

df_orig = df.copy()

# Create lookup frame so we can get the 'Song - Artist' later
item_lookup = df[['song_id','Song - Artist']].drop_duplicates()
item_lookup['song_id'] = item_lookup['song_id'].astype(str)

# Drop 'user' and 'song' and 'Song - Artist'
df = df.drop(['user','song','Song - Artist'], axis=1)

# Create lists of all users, songs, and counts
users = list(np.sort(df['user_id'].unique()))
songs = list(np.sort(df['song_id'].unique()))
play_counts = list(np.sort(df['count']))

# Get the rows and columns for our matrix
user_ = df['user_id'].astype(float)
item_ = df['song_id'].astype(float)

In [4]:
# data_sparse = sp.sparse.csr_matrix((play_counts, (rows, cols)), shape=(len(users), len(songs)))

In [5]:
sp_user_item = sp.sparse.csr_matrix(
    (df['count'].astype(float), (user_, item_)),
    shape=(len(users), len(songs))
)

sp_item_user = sp.sparse.csr_matrix(
    (df['count'].astype(float), (item_, user_)),
    shape=(len(songs), len(users))
)

# Alternating Least Squares

In [6]:
# Hyperparams
factors=30
regularization=0.1
iterations=20
alpha=40

In [7]:
model = implicit.als.AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations=iterations)

In [8]:
data = (sp_item_user * alpha).astype('double')
model.fit(data)

  0%|          | 0/20 [00:00<?, ?it/s]

In [9]:
recs_raw = model.recommend(1, sp_user_item, N=5)
print(recs_raw)

[(2736, 0.7499832), (1722, 0.7386374), (1160, 0.7153151), (2693, 0.65778404), (2629, 0.62132347)]


In [11]:
for song_id , _ in recs_raw:
    print(item_lookup[item_lookup['song_id'] == song_id.astype(str)]['Song - Artist'])

325048    Show You How - The Killers
Name: Song - Artist, dtype: object
232653    Check On It - Beyoncé feat. Slim Thug
Name: Song - Artist, dtype: object
228969    Everyone's At It - Lily Allen
Name: Song - Artist, dtype: object
346705    Valerie - Amy Winehouse
Name: Song - Artist, dtype: object
631587    I Am The Club (Explicit Album Version) - Plies
Name: Song - Artist, dtype: object


# Bayesian Personalized Ranking

https://medium.com/heyjobs-tech/building-recommendation-system-based-bayesian-personalized-ranking-using-tensorflow-2-1-b814d2704130

The missing values in the sparse matrix are **MNAR**. This is because we do not know if the user did not listen to a certain song because the user didn't like the song, was not aware of the song, or for some other reasons unforeseen to us.

In [13]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(
                factors=factors, 
                regularization=regularization,
                iterations=iterations)

To train a model based on Bayes Personalized Ranking, we need to define the triplets of a user, positive item and negative item.

This means we must define what constitutes either a positive item or negative item in the scope of our dataset.

In [ ]:
# Build triplets
